# Orchestrator Agent Tutorial

This tutorial demonstrates how to create an orchestrator agent using the Atomic Agents library. The orchestrator agent will manage tasks and respond to user inputs in rhyming verse.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KennyVaneetvelde/atomic_agents/blob/main/examples/notebooks/orchestrator.ipynb#)


# Prerequisites

Before proceeding with this notebook, it is highly recommended to read up on the basics of the following libraries:

- **Pydantic**: A data validation and settings management library using Python type annotations. You can find more information and documentation at [Pydantic GitHub](https://github.com/pydantic/pydantic).
- **Instructor**: A Python library that simplifies working with structured outputs from large language models (LLMs). It provides a user-friendly API to manage validation, retries, and streaming responses. More details can be found at [Instructor GitHub](https://github.com/jxnl/instructor).

Understanding these libraries will help you make the most of this library.


## Step 1: Install Necessary Packages

First, we need to install the required packages. Run the following command to install `atomic-agents`, `openai`, and `instructor` libraries.

In [ ]:
# Install necessary packages
%pip install atomic-agents openai instructor

## Step 2: Import Libraries

We will import the necessary libraries for creating the orchestrator agent. Each library serves a specific purpose:
- `ChatMemory`: Manages the chat history.
- `BaseChatAgent`: The base class to create a custom chatbot. Can be extended for additional functionality if needed.
- `SystemPromptGenerator` and `SystemPromptInfo`: To define and generate system prompts.
- `BaseTool`, `CalculatorTool`, `SearxNGSearchTool`: Tools that the orchestrator agent can use to perform tasks.

In [ ]:
import os
from typing import List, Type, Union
from pydantic import BaseModel, Field
from rich.console import Console
from atomic_agents.lib.components.chat_memory import ChatMemory
from atomic_agents.agents.base_chat_agent import BaseChatAgent, BaseChatAgentResponse
from atomic_agents.lib.components.system_prompt_generator import SystemPromptContextProviderBase, SystemPromptGenerator, SystemPromptInfo
import instructor
import openai
from atomic_agents.lib.tools.base import BaseTool
from atomic_agents.lib.tools.calculator_tool import CalculatorTool, CalculatorToolSchema
from atomic_agents.lib.tools.search.searx_tool import SearxNGSearchTool, SearxNGSearchToolConfig, SearxNGSearchToolSchema

## Step 3: Define System Prompt Information

We will define the system prompt information including background, steps, and output instructions. This helps the orchestrator agent understand how to respond to user inputs and manage tasks.
In this example, we will define a system prompt that asks the orchestrator agent to respond to user inputs in rhyming verse and manage tasks efficiently.

In [ ]:
search_tool = SearxNGSearchTool(SearxNGSearchToolConfig(base_url=os.getenv('SEARXNG_BASE_URL'), max_results=10))
calculator_tool = CalculatorTool()

class ToolInfoProvider(SystemPromptContextProviderBase):        
    def get_info(self) -> str:
        response = 'The available tools are:\n'
        for tool in [search_tool, calculator_tool]:
            response += f'- {tool.tool_name}: {tool.tool_description}\n'
        return response

# Define system prompt information including background, steps, and output instructions
system_prompt = SystemPromptInfo(
    background=[
        'This assistant is an orchestrator of tasks designed to be helpful and efficient.',
    ],
    steps=[
        'Understand the user\'s input and the current context.',
        'Look at the tools available and determine which to use next based on the current context, user input, and history.',
        'Execute the chosen tool and provide a relevant response to the user.'
    ],
    output_instructions=[
        'Provide helpful and relevant information to assist the user.',
        'Be efficient and effective in task orchestration.',
        'Always answer in rhyming verse.'
    ],
    context_providers={
        'tools': ToolInfoProvider(title='Available tools')
    }
)

# Initialize the system prompt generator with the defined system prompt and dynamic info providers
system_prompt_generator = SystemPromptGenerator(system_prompt)

# Test out the system prompt generator
from rich.markdown import Markdown
from rich.console import Console
Console().print('='*50)
Console().print(Markdown(system_prompt_generator.generate_prompt()))
Console().print('='*50)

## Step 4: Initialize Chat Memory

We will initialize the chat memory to store conversation history and load an initial greeting message.

In [ ]:
memory = ChatMemory()
initial_memory = [
    {'role': 'assistant', 'content': 'How do you do? What can I do for you? Tell me, pray, what is your need today?'}
]
memory.load(initial_memory)

## Step 5: Set Up API Key and Create Orchestrator Agent

To use the **OpenAI** API, you need to set up your API key. You can either enter it directly in the code or set it as an environment variable. Additionally, you can choose other clients such as **Anthropic**, **Mistral**, etc. For a full list, check out the [Instructor library docs](https://github.com/jxnl/instructor).

In [ ]:
##################################################################
# ENTER YOUR API KEY BELOW, OR SET IT AS AN ENVIRONMENT VARIABLE #
##################################################################
API_KEY = ''
if not API_KEY:
    # Get the environment variable
    API_KEY = os.getenv('OPENAI_API_KEY')

if not API_KEY:
    raise ValueError('API key is not set. Please set the API key as a static variable or in the environment variable OPENAI_API_KEY.')

client = instructor.from_openai(openai.OpenAI(api_key=API_KEY))

class ResponseSchema(BaseModel):
    chosen_schema: Union[BaseChatAgentResponse, SearxNGSearchToolSchema, CalculatorToolSchema] = Field(..., description='The response from the chat agent.')
    
    class Config:
        title = 'ResponseSchema'
        description = 'The response schema for the chat agent.'
        json_schema_extra = {
            'title': title,
            'description': description,
        }

# Create a chat agent with the specified model, system prompt generator, and memory
# For all supported clients such as Anthropic & Gemini, have a look at the `instructor` library documentation.
agent = BaseChatAgent(
    client=client, 
    system_prompt_generator=system_prompt_generator,
    model='gpt-3.5-turbo',
    memory=memory,
    output_schema=ResponseSchema
)

# Print the response schema
Console().print(ResponseSchema.model_json_schema())
Console().print('='*50)

## Step 6: Main Chat Loop

We will create a main chat loop for testing the orchestrator agent. This loop will allow you to interact with the chatbot in real-time.
To exit the loop, type `/exit` or `/quit`.

In [ ]:
console = Console()
console.print(f'Agent: {initial_memory[0]["content"]}')

while True:
    user_input = input('You: ')
    if user_input.lower() in ['/exit', '/quit']:
        print('Exiting chat...')
        break

    response = agent.run(user_input)
    console.print(f'Agent: {response}')